# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Couchard Darius - Parent Paul - Donne Stefano

## Pump it Up: Data Mining the Water Table
####  April 29, 2021


# 1) Data Pre-Processing




In [13]:
require(tidyr)
require(dplyr)
require(plyr)
training_set<-read.csv("../Data/TrainingSet/4910797b-ee55-40a7-8668-10efd5c1b960.csv",header=TRUE) # loads the training set csv file (it's magic)
dim(training_set) # dimension of the set 
names(training_set) # names of the variables

traninin_labels<-read.csv("../Data/TrainingLabel/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv", header=TRUE) # Loads the corresponding labels


[1] 59400    40

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group"

## How to enhance the data set :

After having analyzed the data set and assessed each variable relevance, it's needed to standardize the datas.

First, it's needed to remove empty values in the table (Imputation): each NaN or empty cell has to be removed or replaced, different solution exists : 
* Mapping NaN<-0 for nominal categorical variables
* Replace missing value by mean of column for numerical variables

Then, modifications have to be made depending on the nature of the data :
* If a column (variable) consists in continous numerical values : standardization is applied such has obtaining a new column with a mean value of 0 and a standard deviation of 1 (**longitude**)
* If a column is made of ordinal categorical variable (hierarchy between categories) : map each string to a numerical value (**water_quality**)
* In case of nominal categorical variable : apply one hot encoding -> create new column (with binary values) for each category (**source_type**)
<br/>


## Methods are implemented to apply these changes :

In [14]:
# METHOD TO REASSIGN EMPTY VALUES
NaN_handler_categorical <- function(column_name) { # input : column_name (name of the variable)
    training_set[training_set[,column_name] == "", ] <- 0 # select row where column element is empty string "" and assign to 0
    training_set[is.na(training_set[,column_name]),] <- 0 # select row where column element is NaN and assign to 0
    return(training_set)
}


# METHOD TO REASSIGN EMPTY VALUES
NaN_handler_num<- function(column_name) { # input : column_name (name of the variable)
    mean_col <- mean(training_set[,column_name], na.rm = TRUE)
    training_set[is.na(training_set[,column_name]),] <- mean_col
    return(training_set)
}



In [15]:
# STANDARDIZATION METHOD FOR VARIABLES WITH CONTINUOUS NUMERICAL VALUES
Standardization <- function(column_name){ # input : column_name (name of the variable)
    mean_col <- mean(training_set[,column_name], na.rm = TRUE) # mean of the variable
    sd_col <- sd(training_set[,column_name], na.rm = TRUE) # standard deviation of the variable
    training_set[,column_name]<-(training_set[,column_name]-mean_col)/sd_col # apply the transformation
    # now for the whole column : mean = 0 and sd = 1
    return(training_set)
}

In [16]:
# HANDLING OF NOMINAL CATEGORICAL VARIABLES
# before using : change notebook IOPub data rate limit with Jupyter  notebook --NotebookApp.iopub_data_rate_limit=jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
Nom_cat_handle <- function(column_name){
    training_set <- training_set %>% mutate(value = 1)  %>% spread(column_name, value,  fill = 0 )
    return(training_set)
}


## Process the data-set :

In [17]:
# COLUMNS TO DROP 
column_to_drop<-c("wpt_name","amount_tsh","date_recorded","gps_height","num_private","public_meeting","recorded_by",
                 "scheme_name","scheme_management","payment","quantity_group","source_class","subvillage","waterpoint_type",
                 "extraction_type","extraction_type_group","water_quality","source_type")
#TODO : adds others columns to drop
training_set<-training_set[,!(names(training_set) %in% column_to_drop)] # drop the desired column

In [18]:
# MANAGE SIMPLE NOMINAL CATEGORICAL VARIABLE
#TODO : apply one-hot encoding to other needed variables
training_set<-Nom_cat_handle("basin") # apply one-hot-encoding to the basin related column
training_set<-Nom_cat_handle("payment_type")
training_set<-Nom_cat_handle("quality_group")
training_set<-Nom_cat_handle("quantity")
training_set<-Nom_cat_handle("waterpoint_type_group")
training_set<-Nom_cat_handle("extraction_type_class")
training_set<-Nom_cat_handle("source_type")
# ...

ERROR: Error: Can't extract columns that don't exist.
[31mx[39m Column `source_type` doesn't exist.


In [ ]:
# MANAGE ORDINAL CATEGORICAL VARIABLE

# variable PERMIT
# remap True : 1 , False : 0, Missing "" : NA
training_set$permit <- mapvalues(training_set$permit, 
          from=c("True","False",""), 
          to=c(1,0,NA))
training_set <- transform(training_set, permit = as.numeric(permit)) # transform column data type (char to int)
# replace missing value by mean of column
training_set<-NaN_handler_num("permit")
# Standardize Variable
training_set<-Standardization("permit")



In [ ]:
# MANAGE POPULATION VARIABLE
# NaN have to be replaced by mean of region population
# column 1 = region code, column 2 = population mean in this region
region_code_frame <- data.frame("region" = unique(training_set$region_code),"mean_pop" = NA)
for(row in 1: nrow(region_code_frame) ){ 
    sel <-training_set[which(training_set[,"region_code"]==region_code_frame[row,1],),"population"] # select pop row with corresponding region
    region_code_frame[row,"mean_pop"]<-mean(sel[sel!=0],na.rm = TRUE)
}

# TODO : some region population mean are NaN , find a solution (same problem with region and district_code variables)
# temporairement on remplace les NaN par le mean des autres valeurs dans region_code_frame
region_code_frame[which(is.na(region_code_frame[,2])),2]<-mean(region_code_frame[,2],na.rm=TRUE)

# replace NaN value of population by mean region values
index<-which(training_set[,"population"]==0)
for(elem in index){
    training_set[elem,"population"]<-region_code_frame[training_set[elem,"region_code"],2] # replace NaN by their mean region value
}

# Now Standardize Population Variable 
training_set<-Standardization("population")


In [19]:
dim(training_set)
names(training_set)
training_set$population
training_set$permit

[1] 59400    52

[1] "id"                      "funder"                 
 [3] "installer"               "longitude"              
 [5] "latitude"                "region"                 
 [7] "region_code"             "district_code"          
 [9] "lga"                     "ward"                   
[11] "population"              "permit"                 
[13] "construction_year"       "management"             
[15] "management_group"        "source"                 
[17] "Internal"                "Lake Nyasa"             
[19] "Lake Rukwa"              "Lake Tanganyika"        
[21] "Lake Victoria"           "Pangani"                
[23] "Rufiji"                  "Ruvuma / Southern Coast"
[25] "Wami / Ruvu"             "annually"               
[27] "monthly"                 "never pay"              
[29] "on failure"              "per bucket"             
[31] "colored"                 "fluoride"               
[33] "good"                    "milky"                  
[35] "salty"                   "dry"                    
[37] "enough"                  "insufficient"           
[39] "seasonal"                "unknown"                
[41] "cattle trough"           "communal standpipe"     
[43] "dam"                     "hand pump"              
[45] "improved spring"         "gravity"                
[47] "handpump"                "motorpump"              
[49] "other"                   "rope pump"              
[51] "submersible"             "wind-powered"

[1]   109   280   250    58     0     1     0     0     0     0   345   250
   [13]     0     1     0   200    35    50  1000     1     4     0   350   210
   [25]   156   140   260     0     1     1     0   150   180    70   100   200
   [37]     0     0   100   230    50     0   250     1   100     0    30     0
   [49]    70    20    10   200     0     0     0     1    45   150   456     0
   [61]   200     1   567   130   225     1    50     0    54     1   280     0
   [73]     0   200     1     0    75   140   900     0     0   100   360     0
   [85]   544     0   441     0    20     0   120   280    40   221     0   950
   [97]  1430     0     0     0   110   256   150     1    75   320     0     0
  [109]     1     0   120   120     0     0     0     0    90   804   600     1
  [121]     0     0     0   370     0     0    50   305     0     0     0   150
  [133]     0     0     1    50     0     1     0     0     0   150     0     1
  [145]     1   150     0    40   200     0  1600     0    50   400   590     1
  [157]     0     0     0     1   250   450    80   560     0  2500     0  1050
  [169]     0    36   540   300    12   450   632    50   245     0     0   600
  [181]     1    32   120    59   230     1     0     0     0     0     1     0
  [193]     0    50    25   150   500     1   100     0     0     1    40    40
  [205]     0     0     0   400   100   200    20     0   100     0   450     0
  [217]     1     0   150     0    30   140    90   570     1     1   700   111
  [229]     0     0   120   350     1   630   270     0     0     1     0   100
  [241]   250     0     0   200     1     0     1  1000    30   500    55  1700
  [253]     1     0     0     0     0   800     1     1     0   250     0     0
  [265]     0   210   320    87     0  2530     0     0     0     0   450    86
  [277]     1     1    80     1    50   450     0   150   150     0     1     0
  [289]     0   500   450   230    96   150   100     0     1   300     0    36
  [301]     0     0     0     0     0     0     0     0   150    50   900     0
  [313]     0     0   500   350   200   500   150     0   270     0     0     0
  [325]   600     1     0   450    86     0   350    25     0     0     0     1
  [337]     0    75   300   240     0   203     0   200   215     1     0   360
  [349]    98   160   450   200     0  1200     1     0   300     0     0   200
  [361]     0     1     0     0   210   300   400   350     0   120     0   100
  [373]   500   300    50     1     1     0     1     0   400   100   250   450
  [385]   309     0     0     0   100    95     0    60   130     1     1    80
  [397]     0   150   360   250    85   159  1000     0     0     0     0     1
  [409]   130     1     1     0   200     0    70    35     0   100    65   250
  [421]    50     1     1     0     1     1     1     0  1000     0    30     0
  [433]    48     0     0     1   300     0     1     0   250  2150   300     0
  [445]   200  1013     0     0   220     0     1     0    85   210     0    86
  [457]   500    95  1680     0   120    60     0   350     1   210     1   375
  [469]   250  2000     1   125     0   700     0    30  3226   200     1   250
  [481]   250     0   300   200     0   560   150   280     1    50   450     0
  [493]     0   250     0     1   140     0     1    50   150   200     0   140
  [505]     1   400   500   200   253   200   250   450     0   200     0    88
  [517]   820   130     0     1    60     0    40   140     0     0   133     1
  [529]   650     1     0    50   200     0   230     0   163     1     0   150
  [541]     1     0     0   430     0     1   290   150    50     1   100     1
  [553]   100   125  2100     0     0     1     0     0     0   285   300     1
  [565]    40     0   425   100     0   180     1     0     0   155   150    50
  [577]   250    90  1200     1     0   450     1     0     0   500  1500   230
  [589]     0   150   300    80    50   140   185     0   560   230    50   700
  [6

[1] "False" "True"  "True"  "True"  "True"  "True"  "True"  "True"  "True" 
   [10] "True"  "False" "False" "False" "True"  "True"  "False" "True"  "True" 
   [19] "True"  "True"  "False" "False" "True"  "True"  "True"  "True"  "False"
   [28] "True"  "True"  "True"  "True"  "True"  "True"  "False" "False" "True" 
   [37] "True"  "False" "True"  "False" "True"  "False" "True"  ""      "True" 
   [46] "True"  "True"  ""      "True"  "True"  "True"  "True"  "True"  "True" 
   [55] "True"  "True"  "True"  "True"  "True"  "True"  "False" "True"  "True" 
   [64] "True"  "True"  ""      "True"  "True"  "True"  "False" "True"  "False"
   [73] "True"  "True"  "False" "False" "True"  "True"  "False" "True"  "False"
   [82] "True"  "True"  "False" "False" "True"  "True"  "False" "True"  "False"
   [91] "True"  "True"  "True"  "True"  "True"  "True"  "False" "True"  "True" 
  [100] "False" "True"  "True"  "False" "False" "True"  "True"  "False" "True" 
  [109] "True"  ""      "True"  "True"  "True"  "False" "True"  "True"  "True" 
  [118] "False" ""      "True"  "False" "True"  "True"  ""      ""      "True" 
  [127] ""      "True"  "True"  "True"  "True"  "True"  "False" "True"  "True" 
  [136] "False" "False" "False" "True"  "True"  "True"  "True"  "True"  "True" 
  [145] "True"  "False" "True"  "False" "True"  "True"  "True"  "True"  "False"
  [154] "True"  "False" "False" "True"  "True"  "False" "False" "False" "False"
  [163] "True"  "True"  "True"  "False" "False" "True"  "False" "True"  "True" 
  [172] "True"  "True"  "True"  "True"  "True"  "True"  "False" "False" "False"
  [181] "True"  "False" "True"  "False" "False" "True"  "True"  "True"  "False"
  [190] "True"  ""      "True"  "True"  "True"  "True"  "True"  "True"  "True" 
  [199] "True"  "False" "False" ""      "False" "False" "False" "False" "False"
  [208] "False" "True"  "True"  "True"  "True"  "True"  "False" "True"  "True" 
  [217] "True"  "True"  "True"  "False" ""      "True"  "True"  "True"  "True" 
  [226] "False" "True"  "True"  "True"  "True"  "True"  "True"  ""      "False"
  [235] "True"  "True"  "False" "True"  "True"  "True"  "True"  "True"  "False"
  [244] "True"  "True"  "False" "True"  "False" "True"  "False" "True"  "True" 
  [253] "True"  "True"  "False" "True"  "True"  "True"  "True"  "False" "True" 
  [262] "False" "False" "True"  "True"  "True"  "True"  ""      "True"  "True" 
  [271] ""      "True"  "False" "True"  "False" "True"  "True"  "True"  "False"
  [280] "False" "True"  "False" "True"  "True"  "True"  "True"  "True"  "False"
  [289] "True"  "False" "True"  "True"  "True"  "True"  "False" "False" "True" 
  [298] "True"  "False" "True"  "False" "True"  "True"  "True"  "False" "True" 
  [307] "True"  ""      "True"  "False" "True"  "True"  "True"  "True"  "False"
  [316] "False" "False" "True"  "False" "True"  "False" "False" "True"  "False"
  [325] "True"  "True"  "True"  "True"  "True"  "False" "True"  "True"  "False"
  [334] "False" "False" "False" "True"  "True"  "True"  "False" "False" "True" 
  [343] "False" "True"  "True"  "True"  "False" "False" "True"  "True"  "True" 
  [352] "True"  "True"  "True"  "True"  "True"  "True"  "True"  "False" "True" 
  [361] "False" "True"  "False" "False" "True"  "True"  "False" "False" "True" 
  [370] "True"  "True"  "True"  "True"  "True"  "True"  "True"  "False" "True" 
  [379] "False" "False" "True"  "True"  "True"  "False" "True"  "True"  "True" 
  [388] "False" "True"  "True"  "True"  "False" "True"  "False" "True"  "True" 
  [397] "True"  "False" "False" "True"  "True"  "True"  "True"  "True"  "True" 
  [406] "True"  "False" "True"  "True"  "False" "False" "True"  "True"  "True" 
  [415] "False" "False" "True"  "False" "True"  "True"  "True"  "True"  "True" 
  [424] "True"  "True"  "True"  ""      "True"  "False" "True"  "True"  "True" 
  [433] "True"  "False" ""      "True"  "True"  "False" "True"  "True"  "True" 
  [442] "False" "True"  "True"  "True"  "True"  "False" "True"  "True"  ""     
  [4